In [8]:
import numpy as np
import time as T
from joblib import Parallel, delayed
from scipy.interpolate import griddata,interpn,RegularGridInterpolator
import math
import os
import datetime
import rioxarray
import xarray as xr
import multiprocessing
import subprocess
DN_PATH = '/data01/GEO/INPUT/'
target ='/data01/people/liwei/Data/AHI_Validation/'


# site_name = 'ROI 3'
# site = [27,96.5] # ROI 3

# site_name = 'ROI 6'
# site = [46,114]

# site_name = 'ROI 7'
# site = [35.328,99.052]

site_name = 'ROI 8'
site = [42.990,107.043]

# site_name = 'ROI 9'
# site = [41.7,104.6]

# site_name = 'ROI 10'
# site = [49.6,119.4]

In [9]:
d_ahi = []
ws = '/data01/people/liwei/Data/2021_AHI'
with open("/data01/people/liwei/Data/Matching_Result/{}_2021_PL.txt".format(site_name), "r") as f:
    for line in f.readlines():
        line = line.strip('\n') 
        if line[0:8] == 'AHI TIME':
            ahi = [line[9:13],line[14:16],line[17:19],line[20:22],line[23:25]]
            d_ahi.append("".join(ahi))
            
def mkdir(path):
    folder = os.path.exists(path) 
    if not folder:
        os.makedirs(path)

In [10]:
# Reflectance

res = 0.01

u_lat,d_lat = site[0]+0.06,site[0]-0.06
l_lon,r_lon = site[1]-0.06,site[1]+0.06



row_AHI = round((u_lat - d_lat) / res)
col_AHI = round((r_lon - l_lon) / res)

row_u_AHI = round((60 - u_lat)/res)
col_l_AHI = round((l_lon - 85)/res)

def DN2TBB(data):
    LUT=np.loadtxt(DN_PATH + 'count2tbb_v102/vis.03')
    return LUT[data,1]

for date in d_ahi:
    YYYY = date[0:4]
    MM = date[4:6]
    DD = date[6:8]
    HH = date[8:10]
    MIN = date[10:12]
    start_time = T.time()
    time = HH + MIN

    print("start processing {}".format(date))
    # make dir
    folder_AC = target+date+'_AC/'
    mkdir(folder_AC)
    # Download AHI
    with open(ws + '/{}.vis.03.fld.geoss'.format(date),'rb') as fp:
        data = np.frombuffer(fp.read(),dtype='>u2').reshape(12000,12000)
        data = DN2TBB(data)        
    AHI_data = data[row_u_AHI:row_u_AHI + row_AHI , col_l_AHI:col_l_AHI + col_AHI]


    AHI_data_file=open(folder_AC+'/'+date+'_'+site_name+'_b04_toa.dat','wb')
    AHI_data.astype('f4').tofile(AHI_data_file)
    AHI_data_file.close()
#                 remove_original_file(folder_original)
    end_time=T.time()
    TIME=end_time-start_time
    print('time: {:.1f} secs, {:.1f} mins,{:.1f} hours'.format(TIME,TIME/60,TIME/3600))
                    
                    
                    

start processing 202105040400
time: 2.0 secs, 0.0 mins,0.0 hours
start processing 202105190400
time: 0.6 secs, 0.0 mins,0.0 hours
start processing 202106030400
time: 0.6 secs, 0.0 mins,0.0 hours
start processing 202106180410
time: 0.6 secs, 0.0 mins,0.0 hours
start processing 202107300410
time: 0.6 secs, 0.0 mins,0.0 hours
start processing 202108100410
time: 0.6 secs, 0.0 mins,0.0 hours
start processing 202108140400
time: 0.6 secs, 0.0 mins,0.0 hours
start processing 202109020400
time: 0.6 secs, 0.0 mins,0.0 hours


In [16]:
# Radiance

res = 0.005

u_lat,d_lat = site[0]+0.06,site[0]-0.06
l_lon,r_lon = site[1]-0.06,site[1]+0.06


row_AHI = int((u_lat - d_lat) / res)
col_AHI = int((r_lon - l_lon) / res)

row_u_AHI = int((60 - u_lat)/res)
col_l_AHI = int((l_lon - 85)/res)




def DN2TBB(data):
    LUT=np.loadtxt('/media/liwei/Data/count2tbb_v102/ext.01')
    return LUT[data,1]

for date in d_ahi:
    YYYY = date[0:4]
    MM = date[4:6]
    DD = date[6:8]
    HH = date[8:10]
    MIN = date[10:12]
    start_time = T.time()
    time = HH + MIN

    print("start processing {}".format(date))
    # make dir
    folder_original = target+date+'_original/'
    folder_AC = target+date+'_AC/'
    mkdir(folder_original)
    mkdir(folder_AC)
    # Download AHI
    with open('/home/liwei/Downloads/{}.ext.01.fld.geoss'.format(date),'rb') as fp:
        data = np.frombuffer(fp.read(),dtype='>u2').reshape(24000,24000)
        data = DN2TBB(data)        
    AHI_data = data[row_u_AHI:row_u_AHI + 24 , col_l_AHI:col_l_AHI + 24]


    AHI_data_file=open(folder_AC+'/'+date+'_'+site_name+'_b03_toa.dat','wb')
    AHI_data.astype('f4').tofile(AHI_data_file)
    AHI_data_file.close()
#                 remove_original_file(folder_original)
    end_time=T.time()
    TIME=end_time-start_time
    print('time: {:.1f} secs, {:.1f} mins,{:.1f} hours'.format(TIME,TIME/60,TIME/3600))
                    
                    
                    

start processing 202007020410
time: 5.4 secs, 0.1 mins,0.0 hours
start processing 202007130420
time: 4.2 secs, 0.1 mins,0.0 hours
start processing 202007170410
time: 5.3 secs, 0.1 mins,0.0 hours
start processing 202007210410
time: 6.7 secs, 0.1 mins,0.0 hours
start processing 202008010410
time: 4.4 secs, 0.1 mins,0.0 hours
start processing 202008050410
time: 5.5 secs, 0.1 mins,0.0 hours
start processing 202008160350
time: 4.1 secs, 0.1 mins,0.0 hours
start processing 202008200400
time: 5.8 secs, 0.1 mins,0.0 hours
start processing 202008240400
time: 4.1 secs, 0.1 mins,0.0 hours
start processing 202009080340
time: 5.5 secs, 0.1 mins,0.0 hours
start processing 202102250350
time: 4.3 secs, 0.1 mins,0.0 hours
start processing 202103120340
time: 5.8 secs, 0.1 mins,0.0 hours
start processing 202103160330
time: 4.4 secs, 0.1 mins,0.0 hours
start processing 202103270340
time: 5.5 secs, 0.1 mins,0.0 hours
start processing 202103310400
time: 4.3 secs, 0.1 mins,0.0 hours
start processing 20210415